Detects when a student has missed an assignment within n past assignments and then sends reminders

In [ ]:
%cd ~/Dropbox/CanvasHacks

from CanvasHacks import environment

# Widgets
from CanvasHacks.Widgets.SkaaControls import multiple_unit_control
from CanvasHacks.Widgets.LiveSelection import make_test_selector
from CanvasHacks.Widgets.AssignmentSelection import make_unit_chooser, make_selection_button

# Repos
from CanvasHacks.Repositories.overview import SkaaOverviewRepository

# Data store
# Only used as a convenient way to load relevant repositories 
from CanvasHacks.Displays.dashboard import ControlStore

In [ ]:
# TEST = True
TEST = False

# SEND = True
SEND = False

# Late reviews

In [ ]:
from CanvasHacks.StudentNaggers.skaa_review import SkaaReviewNagger

## Send for multiple units

In [ ]:
def run_review_nagger_on_units(unit_numbers):
    """Sends reminder message to students who have not yet done the 
    assigned review for each of the provided units
    :unit_numbers List of integers
    """
    control_store = ControlStore()
    for i in unit_numbers:
        control_store.load_unit(i)
        repo = control_store.skaa_repos[i]
        nagger = SkaaReviewNagger(repo, is_test=TEST, send=SEND)
        print(f"Nagging about late reviews for unit {i}. \nTEST is {TEST}. \nSEND is {SEND}")
        nagger.run()

In [ ]:
units = [1]
# units = [i for i in range(1,7)]

In [ ]:
run_review_nagger_on_units(units)

In [ ]:
# For multiple classes
units = [1]
# units = [i for i in range(1,7)]

# not using its log display features, just a convenient way to load what's needed. 
control_store = ControlStore()
[control_store.load_unit(i) for i in units]

repo = control_store.skaa_repos[environment.CONFIG.unit.unit_number]
nagger = SkaaReviewNagger(repo, is_test=TEST, send=SEND)

print(f"Will run on unit {environment.CONFIG.unit.unit_number}. \nTEST is {TEST}. \nSEND is {SEND}")

## Send for single unit

In [ ]:
# make_test_selector()
make_unit_chooser(num_units=8)

In [ ]:
repo = control_store.skaa_repos[environment.CONFIG.unit.unit_number]
nagger = SkaaReviewNagger(repo, is_test=TEST, send=SEND)
print(f"Nagging about late reviews for unit {environment.CONFIG.unit.unit_number}. \nTEST is {TEST}. \nSEND is {SEND}")


In [ ]:
nagger.run()

# Late essay assignments

In [ ]:
from CanvasHacks.StudentNaggers.essay import EssayNagger

In [ ]:
def run_essay_nagger_on_units(unit_numbers, test=TEST, send=SEND):
    """Sends reminder message to students who have not yet done the 
    assigned essay for each of the provided units
    :unit_numbers List of integers
    """
    # not using its log display features, just a convenient way to load what's needed. 
    control_store = ControlStore()
    for i in unit_numbers:
        control_store.load_unit(i)
        repo = control_store.skaa_repos[i]
        nagger = EssayNagger(repo, is_test=test, send=send)
        print(f"Nagging about late reviews for unit {i}. \nTEST is {test}. \nSEND is {send}")
#         nagger.run()

In [ ]:
units = [1]
# units = [i for i in range(1,7)]


In [ ]:
run_essay_nagger_on_units(units)

In [ ]:
units = [1]
# units = [i for i in range(1,7)]

# not using its log display features, just a convenient way to load what's needed. 
control_store = ControlStore()
[control_store.load_unit(i) for i in units]

In [ ]:

repo = control_store.skaa_repos[environment.CONFIG.unit.unit_number]
nagger = EssayNagger(repo, is_test=TEST, send=SEND)

# Consolidated controls (dev)

In [ ]:
from CanvasHacks.Widgets.NagControls import nag_button_area

store = ControlStore()
nag_button_area(store)

# ------------- Old

In [ ]:
# not using its log display features, just a convenient way to load what's needed. 
# control_store = ControlStore()
# control_store.load_unit(unit=environment.CONFIG.unit)

In [ ]:
adam_id = 6417

# Test data
test_course_id = 85210

main_quiz = 165098
review_quiz = 165820
meta_review = 165821

url = 'https://canvas.csun.edu/api/v1/courses/85210/assignments/{}/submissions'

# last semester data
prev_course_id = 62657
prev_quiz = 151633

COURSE_ID = test_course_id
# COURSE_ID = prev_course_id
# QUIZ_ID = prev_quiz

TEST_STUDENT_ID = 168439

SEMESTER_NAME = 'S20'
LOC = '{}/Box Sync/TEACHING/Phil 305 Business ethics/Phil305 S20'.format(environment.ROOT)# placeholder for where the access codes are stored
ACCESS_CODES_FP = "{}/{}-assignment-access-codes.xlsx".format(LOC, SEMESTER_NAME)

UNIT_NUMBER = 1

In [ ]:
pd.to_datetime('2020-01-22T03:02:56Z')

In [ ]:
# Initialize a Canvas api objects
canvas = Canvas(environment.CONFIG.canvas_url_base, environment.CONFIG.canvas_token)
course = canvas.get_course(COURSE_ID)


In [ ]:
assignments = [ (a.id, pd.to_datetime(a.due_at)) for a in course.get_assignments() if a.due_at]
# make sure they are sorted in expected order
assignments.sort(key=lambda x: x[1])

assignments

In [ ]:
# Assignments due before today
previous_assignments = [a for a in assignments if a[1] < pd.to_datetime(datetime.datetime.now(tzinfo='UTC'))]
previous_assignments

In [ ]:
def s(a):
    return a[1]
assignments


In [ ]:
assignments.sort(key=s, reverse=True)
assignments